<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/L09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Software Development
# Lab 9: Shor's Factorization Algorithm
# Copyright 2024 The MITRE Corporation. All Rights Reserved.

# Note: Use little endian ordering when storing and retrieving integers from
# qubit registers in this lab.

In [26]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, UnitaryGate
from qiskit.visualization import plot_histogram
from math import ceil, log2, pow
import numpy as np

In [22]:
# E01

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    # Shift and multiply by constant
    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    # Swap to target register
    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    # Inverse shift and multiply by inverse constant
    inv_c = pow(c, -1, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

# Quantum modular exponentiation
def exp_mod(circ, a, b, input_qubits, output_qubits):
    # Placeholder for measurement and classical exponentiation
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    # Function to convert classical bit string to integer
    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    # Create a classical computation placeholder
    circ.append(UnitaryGate(np.eye(2**len(output_qubits))), output_qubits)  # Identity operation as a placeholder

    return classical_exp_mod

# Example usage in a circuit
number_to_factor = 15
guess = 7
input_size = ceil(log2(number_to_factor + 1))
output_size = input_size * 2

input_qubits = QuantumRegister(input_size, 'input')
output_qubits = QuantumRegister(output_size, 'output')
classical_bits = ClassicalRegister(input_size + output_size, 'c')

qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

# Apply Hadamard gates to input qubits
qc.h(input_qubits)

# Perform modular exponentiation
exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

# Measure the qubits
qc.measure(input_qubits, classical_bits[:input_size])
qc.measure(output_qubits, classical_bits[input_size:])

# Simulate the circuit
sim = AerSimulator()
t_qc = transpile(qc, sim)
result = sim.run(t_qc).result()
counts = result.get_counts()

print(counts)

{'0100 000000000100': 64, '1100 000000001100': 66, '1000 000000001000': 72, '0101 000000000101': 79, '1110 000000001110': 62, '0110 000000000110': 75, '0001 000000000001': 61, '1101 000000001101': 65, '0111 000000000111': 67, '1010 000000001010': 56, '1001 000000001001': 67, '0010 000000000010': 43, '0000 000000000000': 45, '1111 000000001111': 68, '0011 000000000011': 68, '1011 000000001011': 66}


In [38]:
# E02

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    inv_c = mod_inverse(c, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

def exp_mod(circ, a, b, input_qubits, output_qubits):
    modular_multiply_by_constant(circ, b, a, output_qubits)

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    qc.h(input_qubits)

    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    qc.append(QFT(len(input_qubits), inverse=True).to_instruction(), input_qubits)

    qc.measure(input_qubits, classical_bits)

    return qc

def run_subroutine_test(number_to_factor, guess, period, tolerance):
    n = ceil(log2(number_to_factor + 1))
    qc = find_approx_period(number_to_factor, guess, n)

    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc, shots=1024).result()
    counts = result.get_counts()

    measured_value = max(counts, key=counts.get)
    measurement = int(measured_value, 2)

    search_space = 2 ** n
    scaled_measurement = measurement / search_space * period
    nearest_multiple = round(scaled_measurement)
    delta = abs(scaled_measurement - nearest_multiple)

    print(f"Measured {measurement}/{search_space} => {scaled_measurement}, delta = {delta}")

    return delta < tolerance

    # Measure the output qubits
    qc.measure(output_qubits, classical_bits)

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc).result()
    counts = result.get_counts()

    print(counts)
    return counts

# Example usage
number_to_factor = 15
guess = 7
n = ceil(log2(number_to_factor + 1))

find_approx_period(number_to_factor, guess, n)

In [27]:
# E02


def inverse_qft(qc, qubits):
    qc.append(QFT(len(qubits), inverse=True).to_instruction(), qubits)

def exp_mod(circ, a, b, input_qubits, output_qubits):
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    circ.append(UnitaryGate(np.eye(2**len(output_qubits))), output_qubits)  # Identity operation as a placeholder

    return classical_exp_mod

def find_approx_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Apply the inverse QFT to input qubits
    inverse_qft(qc, input_qubits)

    # Measure the input qubits
    qc.measure(input_qubits, classical_bits)

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc).result()
    counts = result.get_counts()

    print(counts)
    return counts

# Example usage
number_to_factor = 15
guess = 7
n = ceil(log2(number_to_factor + 1))

find_approx_period(number_to_factor, guess, n)

{'0111 1100': 1, '0010 1110': 3, '0000 0100': 1, '1110 1100': 7, '1000 1011': 4, '1001 0001': 3, '0001 0001': 1, '0111 0101': 3, '0100 1100': 2, '1100 0011': 2, '0001 1000': 3, '0111 1111': 3, '0101 1110': 4, '1100 1100': 1, '0111 0001': 1, '0110 0101': 3, '0000 1110': 1, '1000 0000': 1, '1101 0101': 2, '0001 1111': 3, '1110 1001': 2, '0101 0100': 2, '0101 1100': 3, '1100 1010': 7, '0010 1010': 1, '1100 0100': 3, '1011 0001': 1, '1110 1010': 2, '0011 1101': 3, '1101 1100': 3, '0100 1111': 4, '1100 0110': 5, '0101 0000': 2, '0001 1110': 4, '1111 1110': 4, '1001 0100': 1, '1001 1100': 4, '0000 0000': 3, '0001 1100': 2, '0010 0010': 6, '1111 0011': 5, '1100 1011': 2, '1110 0000': 4, '0111 0000': 6, '1100 1101': 4, '1111 0101': 3, '1110 0010': 4, '0010 0110': 9, '0100 0010': 4, '1110 0111': 2, '0011 0111': 4, '1011 1011': 6, '1001 1011': 3, '0001 0100': 3, '1010 1100': 4, '1000 0100': 4, '0010 0111': 2, '1000 0101': 4, '1000 0001': 2, '1000 1010': 8, '1010 1001': 4, '0100 0110': 2, '0101 1

{'0111 1100': 1,
 '0010 1110': 3,
 '0000 0100': 1,
 '1110 1100': 7,
 '1000 1011': 4,
 '1001 0001': 3,
 '0001 0001': 1,
 '0111 0101': 3,
 '0100 1100': 2,
 '1100 0011': 2,
 '0001 1000': 3,
 '0111 1111': 3,
 '0101 1110': 4,
 '1100 1100': 1,
 '0111 0001': 1,
 '0110 0101': 3,
 '0000 1110': 1,
 '1000 0000': 1,
 '1101 0101': 2,
 '0001 1111': 3,
 '1110 1001': 2,
 '0101 0100': 2,
 '0101 1100': 3,
 '1100 1010': 7,
 '0010 1010': 1,
 '1100 0100': 3,
 '1011 0001': 1,
 '1110 1010': 2,
 '0011 1101': 3,
 '1101 1100': 3,
 '0100 1111': 4,
 '1100 0110': 5,
 '0101 0000': 2,
 '0001 1110': 4,
 '1111 1110': 4,
 '1001 0100': 1,
 '1001 1100': 4,
 '0000 0000': 3,
 '0001 1100': 2,
 '0010 0010': 6,
 '1111 0011': 5,
 '1100 1011': 2,
 '1110 0000': 4,
 '0111 0000': 6,
 '1100 1101': 4,
 '1111 0101': 3,
 '1110 0010': 4,
 '0010 0110': 9,
 '0100 0010': 4,
 '1110 0111': 2,
 '0011 0111': 4,
 '1011 1011': 6,
 '1001 1011': 3,
 '0001 0100': 3,
 '1010 1100': 4,
 '1000 0100': 4,
 '0010 0111': 2,
 '1000 0101': 4,
 '1000 0001': 

In [31]:
# E03

from sympy import Rational
from sympy.ntheory.continued_fraction import continued_fraction_periodic, continued_fraction_convergents

def find_period_candidate(numerator, denominator, denominator_threshold):
    fraction = Rational(numerator, denominator)
    continued_fraction = continued_fraction_periodic(fraction.p, fraction.q)
    convergents = continued_fraction_convergents(continued_fraction)

    candidate = None
    for convergent in convergents:
        if convergent.q < denominator_threshold:
            candidate = convergent
        else:
            break
    return (candidate.p, candidate.q)

# Example usage
numerator = 43
denominator = 256
denominator_threshold = 100

candidate = find_period_candidate(numerator, denominator, denominator_threshold)
print(f"Period candidate: p = {candidate[0]}, q = {candidate[1]}")

Period candidate: p = 1, q = 6


In [35]:
# E04

from sympy import mod_inverse

def modular_multiply_by_constant(circ, modulus, c, y):
    n = len(y)
    qs = QuantumRegister(n)
    circ.add_register(qs)

    # Shift and multiply by constant
    for idx in range(n):
        shifted_c = (c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[idx])

    # Swap to target register
    for idx in range(n):
        circ.swap(y[idx], qs[idx])

    # Inverse shift and multiply by inverse constant
    inv_c = mod_inverse(c, modulus)
    for idx in range(n):
        shifted_c = (inv_c << idx) % modulus
        if shifted_c != 0:
            circ.mcx([y[idx]], qs[(idx + shifted_c) % n])

def exp_mod(circ, a, b, input_qubits, output_qubits):
    classical_bits = ClassicalRegister(len(input_qubits), 'm')
    circ.add_register(classical_bits)

    circ.measure(input_qubits, classical_bits)

    def classical_exp_mod(a, bits, b):
        bitstring = ''.join(map(str, bits[::-1]))
        exponent = int(bitstring, 2)
        return pow(a, exponent, b)

    modular_multiply_by_constant(circ, b, a, output_qubits)
    return classical_exp_mod

def find_period(number_to_factor, guess, n):
    input_qubits = QuantumRegister(n, 'input')
    output_qubits = QuantumRegister(n, 'output')
    classical_bits = ClassicalRegister(n, 'classical')
    qc = QuantumCircuit(input_qubits, output_qubits, classical_bits)

    # Apply Hadamard gates to input qubits
    qc.h(input_qubits)

    # Apply the exp_mod circuit (modular exponentiation)
    exp_mod(qc, guess, number_to_factor, input_qubits, output_qubits)

    # Measure the output qubits
    qc.measure(output_qubits, classical_bits)

    # Simulate the circuit
    sim = AerSimulator()
    t_qc = transpile(qc, sim)
    result = sim.run(t_qc).result()
    counts = result.get_counts()

    print("Counts:", counts)
    return counts

# Example usage
number_to_factor = 15
guess = 7
n = ceil(log2(number_to_factor + 1))

find_period(number_to_factor, guess, n)

Counts: {'1011 0000': 56, '0100 0000': 65, '0101 0000': 76, '1010 0000': 57, '1100 0000': 77, '0000 0000': 69, '0111 0000': 52, '1000 0000': 76, '1111 0000': 49, '1001 0000': 67, '1110 0000': 71, '0001 0000': 59, '0010 0000': 54, '0011 0000': 64, '1101 0000': 62, '0110 0000': 70}


{'1011 0000': 56,
 '0100 0000': 65,
 '0101 0000': 76,
 '1010 0000': 57,
 '1100 0000': 77,
 '0000 0000': 69,
 '0111 0000': 52,
 '1000 0000': 76,
 '1111 0000': 49,
 '1001 0000': 67,
 '1110 0000': 71,
 '0001 0000': 59,
 '0010 0000': 54,
 '0011 0000': 64,
 '1101 0000': 62,
 '0110 0000': 70}

In [40]:
# E05

from math import gcd

def modular_exponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2) == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def find_factor(number_to_factor, guess, period):
    if period % 2 != 0:
        # Period is odd, return -1
        return -1

    # Calculate guess^(period/2) % number_to_factor
    x = modular_exponentiation(guess, period // 2, number_to_factor)

    # If x is equivalent to -1 modulo number_to_factor, this period doesn't work for factoring
    if x == number_to_factor - 1:
        return -2

    # Calculate the gcd of (x - 1) and number_to_factor
    factor1 = gcd(x - 1, number_to_factor)
    if factor1 > 1 and factor1 < number_to_factor:
        return factor1

    # Calculate the gcd of (x + 1) and number_to_factor
    factor2 = gcd(x + 1, number_to_factor)
    if factor2 > 1 and factor2 < number_to_factor:
        return factor2

    # If neither factor is found, return -2
    return -2

# Example usage
number_to_factor = 15
guess = 7
period = 4

factor = find_factor(number_to_factor, guess, period)
print(f"Factor found: {factor}")


Factor found: 3
